In [125]:
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [126]:
bestandsnaam = 'Spotify_Youtube.csv'
df = pd.read_csv(bestandsnaam, sep=',', error_bad_lines=False)

/var/folders/rk/6k3gdwwd7470n5mn_07yrgkc0000gn/T/ipykernel_14795/1142002548.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(bestandsnaam, sep=',', error_bad_lines=False)


In [127]:
print(df.columns.tolist())

['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type', 'Uri', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Url_youtube', 'Title', 'Channel', 'Views', 'Likes', 'Comments', 'Description', 'Licensed', 'official_video', 'Stream']


In [128]:
# Toevoegen van twee nieuwe features
df["Time Streamed (hours)"] = df["Duration_ms"] * df["Stream"] / (1000 * 60 * 60)
df["Time Streamed (hours)"] = df["Time Streamed (hours)"].round(decimals=1)
df["Like Percentage"] = (df["Likes"] / df["Views"]) * 100

In [129]:
# Data preprocessing
# Dit zijn de waardes waar we onze voorspellingen op kunnen baseren
df['Views'].fillna(df['Views'].mean(), inplace=True)
df['Duration_ms'].fillna(df['Duration_ms'].mean(), inplace=True)
df['Comments'].fillna(df['Comments'].mean(), inplace=True)
df['Likes'].fillna(df['Likes'].mean(), inplace=True)
df['Danceability'].fillna(df['Danceability'].mean(), inplace=True)
df['Energy'].fillna(df['Energy'].mean(), inplace=True)
df['Key'].fillna(df['Key'].mean(), inplace=True)
df['Loudness'].fillna(df['Loudness'].mean(), inplace=True)
df['Speechiness'].fillna(df['Speechiness'].mean(), inplace=True)
df['Acousticness'].fillna(df['Acousticness'].mean(), inplace=True)
df['Instrumentalness'].fillna(df['Instrumentalness'].mean(), inplace=True)
df['Valence'].fillna(df['Valence'].mean(), inplace=True)
df['Tempo'].fillna(df['Tempo'].mean(), inplace=True)
df['Licensed'].fillna(df['Licensed'].mean(), inplace=True)
df['official_video'].fillna(df['official_video'].mean(), inplace=True)
df['Stream'].fillna(df['Stream'].mean(), inplace=True)

# Dit zijn eigenlijk True/False waardes, maar die kunnen we omzetten naar int en op die manier ook gebruiken om mee te voorspellen
df['Licensed'] = df['Licensed'].astype(int)
df['official_video'] = df['official_video'].astype(int)

# Dit willen we voorspellen, waardes die ontbreken vervangen we door 0 om zo het verschil te kunnen zien tussen wat wij voorspellen en of het eventueel ontbreekt.
selected_columns = ['Views', 'Duration_ms', 'Comments', 'Likes', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Valence', 'Tempo', 'Licensed', 'official_video']

print(df[selected_columns].head(10))


         Views  Duration_ms  Comments      Likes  Danceability  Energy   Key  \
0  693555221.0     222640.0  169907.0  6220896.0         0.818   0.705   6.0   
1   72011645.0     200173.0   31003.0  1079128.0         0.676   0.703   8.0   
2    8435055.0     215150.0    7399.0   282142.0         0.695   0.923   1.0   
3  211754952.0     233867.0   55229.0  1788577.0         0.689   0.739   2.0   
4  618480958.0     340920.0  155930.0  6197318.0         0.663   0.694  10.0   
5  259021161.0     245000.0   72008.0  1844658.0         0.760   0.891  11.0   
6     451996.0     274142.0     241.0    11686.0         0.716   0.897   4.0   
7    1010982.0     209560.0     260.0    17675.0         0.726   0.815  11.0   
8   24459820.0     213750.0   20296.0   739527.0         0.741   0.913   2.0   
9  154761056.0     230426.0   39240.0  1386920.0         0.625   0.877  10.0   

   Loudness  Speechiness  Acousticness  Instrumentalness  Valence    Tempo  \
0    -6.679       0.1770      0.008360   

In [130]:
# Het schalen van de data die erg groot is
scaler = StandardScaler()
df[['Views', 'Comments', 'Likes', 'Duration_ms']] = scaler.fit_transform(df[['Views', 'Comments', 'Likes', 'Duration_ms']])

selected_columns = ['Views', 'Duration_ms', 'Comments', 'Likes', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Valence', 'Tempo', 'Licensed', 'official_video', 'Stream']

print(df[selected_columns].head(10))

df = df[selected_columns]

      Views  Duration_ms  Comments     Likes  Danceability  Energy   Key  \
0  2.208499    -0.016650  0.747216  3.147394         0.818   0.705   6.0   
1 -0.080758    -0.196700  0.018283  0.235471         0.676   0.703   8.0   
2 -0.314922    -0.076675 -0.105585 -0.215883         0.695   0.923   1.0   
3  0.433942     0.073324  0.145415  0.637252         0.689   0.739   2.0   
4  1.931987     0.931247  0.673868  3.134041         0.663   0.694  10.0   
5  0.608032     0.162543  0.233467  0.669012         0.760   0.891  11.0   
6 -0.344325     0.396088 -0.143148 -0.369050         0.716   0.897   4.0   
7 -0.342266    -0.121473 -0.143048 -0.365658         0.726   0.815  11.0   
8 -0.255900    -0.087894 -0.037904  0.043146         0.741   0.913   2.0   
9  0.224023     0.045747  0.061509  0.409782         0.625   0.877  10.0   

   Loudness  Speechiness  Acousticness  Instrumentalness  Valence    Tempo  \
0    -6.679       0.1770      0.008360          0.002330    0.772  138.559   
1    -5

In [131]:
# Kenmerken (features) en doelvariabele (target) selecteren
X = df[['Views', 'Duration_ms', 'Comments', 'Likes', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Valence', 'Tempo', 'Licensed', 'official_video']
].values
y = df['Stream'].values

# Trainingsdata en testdata splitsen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineair regressiemodel initialiseren en trainen
model = LinearRegression()
model.fit(X_train, y_train)

# Voorspellingen maken op de testdata
y_pred = model.predict(X_test)


In [132]:
mse = mean_squared_error(y_test, y_pred)
print('Gemiddelde kwadratische fout (MSE):', mse)

mae = mean_absolute_error(y_test, y_pred)
print('Gemiddelde absolute fout (MAE):', mae)

r2 = r2_score(y_test, y_pred)
print('Coëfficiënt van determinatie (R^2):', r2)


Gemiddelde kwadratische fout (MSE): 2.9299637696201684e+16
Gemiddelde absolute fout (MAE): 96378752.78852002
Coëfficiënt van determinatie (R^2): 0.4661708785876377


In [133]:
# Kenmerken (features) en doelvariabele (target) selecteren
X = df[['Views', 'Duration_ms', 'Comments', 'Likes', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Valence', 'Tempo', 'Licensed', 'official_video']].values
y = df['Stream'].values

# Lijsten voor de scores initialiseren
mse_scores = []
mae_scores = []
r2_scores = []

# Loop over elke variabele en bereken de scores
for i, feature in enumerate(df.columns[:-1]):
    # Kenmerk (feature) selecteren
    X_feature = X[:, i].reshape(-1, 1)

    # Trainingsdata en testdata splitsen
    X_train, X_test, y_train, y_test = train_test_split(X_feature, y, test_size=0.2, random_state=42)

    # Lineair regressiemodel initialiseren en trainen
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Voorspellingen maken op de testdata
    y_pred = model.predict(X_test)

    # Scores berekenen
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Scores toevoegen aan de lijsten
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)

# Sorteren op basis van de MSE-scores in oplopende volgorde
sorted_scores = sorted(zip(df.columns[:-1], mse_scores, mae_scores, r2_scores), key=lambda x: x[1])

# Top 10 resultaten opslaan in een .txt-bestand
with open('resultaten.txt', 'w') as file:
    file.write("Top 10 geselecteerde variabelen:\n")
    for feature, mse, mae, r2 in sorted_scores[:10]:
        file.write(f"Variabele: {feature}\n")
        file.write(f"MSE: {mse:.2f}\n")
        file.write(f"MAE: {mae:.2f}\n")
        file.write(f"R^2: {r2:.2f}\n")
        file.write("\n")


In [134]:
# Kenmerken (features) en doelvariabele (target) selecteren
X = df[['Views', 'Comments', 'Likes']
].values
y = df['Stream'].values

# Trainingsdata en testdata splitsen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineair regressiemodel initialiseren en trainen
model = LinearRegression()
model.fit(X_train, y_train)

# Voorspellingen maken op de testdata
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print('Gemiddelde kwadratische fout (MSE):', mse)

r2 = r2_score(y_test, y_pred)
print('Coëfficiënt van determinatie (R^2):', r2)

Gemiddelde kwadratische fout (MSE): 2.971279631864412e+16
Coëfficiënt van determinatie (R^2): 0.4586432734100836


In [138]:
# Kenmerken (features) en doelvariabele (target) selecteren
X = df[['Views', 'Comments', 'Likes']].values
y = df['Stream'].values

# Maak de pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Trainingsdata en testdata splitsen en model trainen
pipeline.fit(X_train, y_train)

# Voorspellingen maken op de testdata
y_pred = pipeline.predict(X_test)

# Berekenen van de MSE en R^2 scores
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Afdrukken van de scores
print('Gemiddelde kwadratische fout (MSE):', mse)
print('Coëfficiënt van determinatie (R^2):', r2)


Gemiddelde kwadratische fout (MSE): 2.971279631864412e+16
Coëfficiënt van determinatie (R^2): 0.4586432734100836
